In [1]:
from langchain import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain.chains import RetrievalQA
from langchain_pinecone import PineconeVectorStore

c:\Users\McQue\.conda\envs\GPU\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import os
import pinecone
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
PINECONE_API_HOST = os.environ.get('PINECONE_API_ENV')

from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=PINECONE_API_KEY)

In [4]:
##Creating the Index in Pinecone
index_name = "medical-chat-bot"  # change if desired
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

In [12]:
##Configuring the pinecone index
# import time
# if index_name not in existing_indexes:
#     pc.create_index(
#         name=index_name,
#         dimension=384,
#         metric="cosine",
#         spec=ServerlessSpec(cloud="aws", region="us-east-1"),
#     )
#     while not pc.describe_index(index_name).status["ready"]:
#         time.sleep(1)

# index = pc.Index(index_name)

In [5]:
def load_pdf(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()

    return documents

In [6]:
extracted_data = load_pdf("data/")

In [14]:
#extracted_data

In [7]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 2
    )
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)

In [5]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"

In [6]:
#download embedding model
def download_hugging_face_embeddings(model_name):
    embeddings = HuggingFaceEmbeddings(model_name = model_name)
    return embeddings

In [7]:
embeddings = download_hugging_face_embeddings(model_name)

c:\Users\McQue\.conda\envs\GPU\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [29]:
#embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [33]:

# result = embeddings.embed_query("how are you")
# len(result)

In [36]:
from langchain_pinecone import PineconeVectorStore
index_name = "medical-chat-bot"
docsearch = PineconeVectorStore.from_documents(text_chunks, embeddings, index_name=index_name)

KeyboardInterrupt: 

In [8]:
vectorstore = PineconeVectorStore(index_name=index_name, embedding=embeddings)


In [9]:
query = "what are allergies"
retriever = vectorstore.as_retriever(search_type="mmr")
matched_docs = retriever.invoke(query)
for i, d in enumerate(matched_docs):
    print(f"\n## Document {i+1}\n")
    print(d.page_content)


## Document 1

American Academy of Ophthalmology. 655 Beach Street, PO
Box 7424, San Francisco, CA 94120-7424. <http://www.eyenet.org>.KEY TERMS
Allergen —A substance capable of inducing an
allergic response.
Allergic reaction —An immune system reaction to
a substance in the environment; symptomsinclude rash, inflammation, sneezing, itchy wateryeyes, and runny nose.
Conjunctiva —The mucous membrane that covers
the white part of the eyes and lines the eyelids.
Edema —A condition where tissues contain exces-

## Document 2

aller-gies and hypersensitivity to foods, chemicals, and otheragents. Other tests for food allergies are the eliminationand rotation diets, in which foods are systematically eval-uated to determine the ones that are causing problems.

## Document 3

causes much more severe symptoms and generally afever. Allergies to molds or pollens also can make the
nose run. Allergies are usually more persistent than thecommon cold. An allergist can do tests to determine ifthe cold

In [10]:
prompt_template = """
Use the following pieces of information to answer the user's question .
If you don't know the answer, just say you don't knwo, don't try to make up an answer

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else
Helpful answer:
"""

In [11]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}

In [12]:
llm = CTransformers(model=r"model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type="llama",
                    config={"max_new_tokens" : 512,
                            'temperature': 0.8}
                    )

In [13]:
qa = RetrievalQA.from_chain_type(llm = llm,
                                 chain_type="stuff",
                                 retriever = vectorstore.as_retriever(search_kwarg = {'k': 2}),
                                 return_source_documents = True,
                                 chain_type_kwargs= chain_type_kwargs
                                 )

In [14]:
qa.invoke(query)

{'query': 'what are allergies',
 'result': 'I don\'s will contain words:\nI\'s about "I\'s:\nI can be based on how to user\'s:\nAmerican Academy of all the users will be given by Dr. You can be found! Thank you can be here, thanks to your question:\nI don\'s: The American Academy of related to user wants to the user: To allergic substances\nAmerican Academy Of the question 1.\nI amy: Sorry about "I\'s\n\nWe regretfully, but not found!\nThe American Academy of this user might include:\n\n\nAller 1.\nI don\'s\nThank you provide: The American Academy of the user question. I gotteng\n\n\n\n\n\n\nUnfortunately, but not available for the user can befor the user questions? I am not found! Thank you can be given by Eyeahol\nThe American Academy of course.\nI don\'s in terms:\nI can also available:\nThank you provided by jutiful answers are highlights\n\n\nI don\'s\nI don\'s:\nI don\'s\nAller:\nI cannot be found for a user\n\nA combination\nI don\'s to the user\nThe American Academy of similar 

In [22]:
import sys
while True:
    user_input = input(f'input Prompt: ')
    if user_input == 'exit':
        print('Exiting....')
        sys.exit()
    if user_input == '':
        continue
    result = qa.invoke({'query': user_input})
    print(f"query: {result['query']} \nanswer: {result['result']}")